![title](./img/titanic_meme.jpg)

# Explaining model predictions with Shapley values - Random Forest

The notebook below is a modified version of the original by Mike Allen's [Titanic Notebooks](https://github.com/MichaelAllen1966/titanic).

Shapley values provide an estimate of how much any particular feature influences the model decision (prediction). When Shapley values are averaged they provide a measure of the overall influence of a feature.

Shapley values may be used across model types, and so provide a *model-agnostic* measure of a feature's influence. This means that the influence of features may be compared across model types, and it allows *black box* models like neural networks to be explained, at least in part.

Here we will demonstrate Shapley values with random forests.

For more on Shapley values in general see Chris Molner's excellent book chapter:

https://christophm.github.io/interpretable-ml-book/shapley.html

The `shap` package is installed if you have used the Titanic environment yasml file, but otherwise may be installed with `pip install shap`.

More information on the `shap` library, inclusiong lots of useful examples may be found at: https://shap.readthedocs.io/en/latest/index.html

Here we provide an example of using `shap` with Random Forests.

Shap values are returned in a slightly different way to logistic regression  - there is a set of Shap values for each classification probablility ('not survive', 'survive') so we need slightly different syntax to access and use the Shap values.

## Load data and fit model

### Load modules

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Import machine learning methods
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Import shap for shapley values
import shap # `! pip install shap` if neeed

### Load data

The section below downloads pre-processed data, and saves it to a subfolder (from where this code is run).
If data has already been downloaded that cell may be skipped.

Code that was used to pre-process the data ready for machine learning may be found at:
https://github.com/MichaelAllen1966/1804_python_healthcare/blob/master/titanic/01_preprocessing.ipynb

In [ ]:
download_required = True

if download_required:
    
    # Download processed data:
    address = 'https://raw.githubusercontent.com/MichaelAllen1966/' + \
                '1804_python_healthcare/master/titanic/data/processed_data.csv'
    
    data = pd.read_csv(address)

    # Create a data subfolder if one does not already exist
    import os
    data_directory ='./data/'
    if not os.path.exists(data_directory):
        os.makedirs(data_directory)

    # Save data
    data.to_csv(data_directory + 'processed_data.csv', index=False)

In [ ]:
data = pd.read_csv('data/processed_data.csv')
# Make all data 'float' type
data = data.astype(float)

### Divide into X (features) and y (labels)

We will separate out our features (the data we use to make a prediction) from our label (what we are truing to predict).
By convention our features are called `X` (usually upper case to denote multiple features), and the label (survived or not) `y`.

In [ ]:
# Use `survived` field as y, and drop for X
y = data['Survived'] # y = 'survived' column from 'data'
X = data.drop('Survived',axis=1) # X = all 'data' except the 'survived' column

# Drop PassengerId
X.drop('PassengerId',axis=1, inplace=True)

### Divide into training and tets sets

When we test a machine learning model we should always test it on data that has not been used to train the model.
We will use sklearn's `train_test_split` method to randomly split the data: 75% for training, and 25% for testing.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=42,
                                                    test_size=0.25)

### Fit Random Forest model

In [ ]:
model = RandomForestClassifier(n_estimators=100,
                               n_jobs=-1,
                               class_weight='balanced',
                               random_state=42)

model.fit(X_train, y_train)

### Predict values and get probabilities of survival

Now we can use the trained model to predict survival. We will test the accuracy of both the training and test data sets.

In [ ]:
# Predict training and test set labels
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# Predict probabilities of survival
y_prob_train = model.predict_proba(X_train)
y_prob_test = model.predict_proba(X_test)

### Calculate accuracy

In this example we will measure accuracy simply as the proportion of passengers where we make the correct prediction. In a later notebook we will look at other measures of accuracy which explore false positives and false negatives in more detail.

In [ ]:
accuracy_train = np.mean(y_pred_train == y_train)
accuracy_test = np.mean(y_pred_test == y_test)

print (f'Accuracy of predicting training data = {accuracy_train:0.3f}')
print (f'Accuracy of predicting test data = {accuracy_test:0.3f}')

## Examining the model importances

In [ ]:
features = list(X_train)

feature_importances = model.feature_importances_

importances = pd.DataFrame(index=features)
importances['importance'] = feature_importances
importances['rank'] = importances['importance'].rank(ascending=False).values
importances.sort_values('rank').head()

The three most influential features are:

* *male*
* *Fare*
* *age*

Note: random forest importances do not tell us anything about the direction of effect of features (as with random forests, the direction of effect may depend on the value oif other features).

## Get Shapley values

We use the `shap_values` method from the SHAP library to get Shapley values.
We use the `explainer` method from the SHAP library to get Shapley values along with other data.

We pass a sample to the explainer to speed up Shap (which can be slow with random forests - these values are used as expected baseline values for features).

[`check_additivity`](https://shap-lrjball.readthedocs.io/en/latest/generated/shap.TreeExplainer.html?highlight=check_additivity) has been disabled as the fit reported a small differnce between between predicted probability and baseline probability with all Shap values summed.

In [ ]:
# Get list of features
features = list(X_train)

# Train explainer on Training set
explainer = shap.TreeExplainer(model,
                               X_train.sample(100))
    
# Get Shap values (extended version has other data returned as well as shap values)
shapley_values_train_extended = explainer(X_train, check_additivity=False)
shapley_values_train = shapley_values_train_extended.values[:,:,1]

shapley_values_test_extended = explainer(X_test, check_additivity=False)
shapley_values_test = shapley_values_test_extended.values[:,:,1]

# Calculate mean Shapley value for each feature in training set
importances['mean_shapley_values'] = np.mean(shapley_values_train, axis=0)

# Calculate mean absolute Shapley value for each feature in training set
# This will give us the average importance of each feature
importances['mean_abs_shapley_values'] = np.mean(
    np.abs(shapley_values_train),axis=0)

Add Shapley values to coefficient table.

In [ ]:
importances.sort_values(by='importance', ascending=False).head()
importances.head(10)

Get top 10 influenctial features by co-effieceints or Shapley

In [ ]:
# Get top 10 features
importance_top_10 = \
    importances.sort_values(by='importance', ascending=False).head(10).index
shapley_top_10 = \
    importances.sort_values(
    by='mean_abs_shapley_values', ascending=False).head(10).index

# Add to DataFrame
top_10_features = pd.DataFrame()
top_10_features['importances'] = importance_top_10.values
top_10_features['Shapley'] = shapley_top_10.values

# Display
top_10_features

We can see a lot of overlap between the most import fatures as estimated by coefficients and those estimated using mean absolute Shapley values. But they are not identical.

Plot comparison of Shapley and model coefficients:

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111)

# Plot points
x = importances['importance']
y = importances['mean_abs_shapley_values']

ax.scatter(x, y)
ax.set_title('Shapley value vs model weight (coefficient) for each feature')
ax.set_ylabel('Mean absolute Shapley value')
ax.set_xlabel('Feature importance')

plt.grid()
plt.show()

## Plots of Shapley values

### [Summary Plot](https://shap-lrjball.readthedocs.io/en/latest/generated/shap.summary_plot.html)

The `summary_plot` using a `plot_type` option of `bar` gives us the overall importance of each feature across the population.

Here we limit the num,ber of features shown to 15 (default is 20).

In [ ]:
fig = plt.figure(figsize=(6,6))

shap.summary_plot(shap_values = shapley_values_train, 
                  features = X_train.values,
                  feature_names = X_train.columns.values,
                  plot_type='bar',
                  max_display=15,
                  show=False)
plt.tight_layout()
plt.show()

### Summary Plot - Beeswarm

Without specifying a `plot_type` option of `bar`, `summary_plot` gives us a *beeswarm* plot, showing the Shapley values for all instances.

In [ ]:
fig = plt.figure(figsize=(6,6))

shap.summary_plot(shap_values = shapley_values_train, 
                  features = X_train.values,
                  feature_names = X_train.columns.values,
                  max_display=15,
                  show=False)
plt.tight_layout()
plt.show()

### [Waterfall Plot](https://shap-lrjball.readthedocs.io/en/latest/generated/shap.waterfall_plot.html?highlight=plots.waterfall)

Waterfall plots show the influence of  individual features on model prediction. These are shown as the effect on [*log odds ratio*](https://www.statisticshowto.com/log-odds/) of survival. *Log odds ratio* are usually shown as these are additive, whereas probabilities are not.

Waterfall plots put the most influential features at the top.

Get locations of passengers with low or high probability of survival.

In [ ]:
y_prob = model.predict_proba(X_test)[:,1]

# Get the location of an example each where porbability of survival
# is <0.01 or >0.99

location_low_probability = np.where(y_prob < 0.05)[0][0]
location_high_probability = np.where(y_prob > 0.95)[0][0]

Waterfall plot for passenger with lowest probability of survival:

In [ ]:
shap.plots.waterfall(shapley_values_test_extended[location_low_probability][:,1], 
                     max_display=15)

Waterfall plot for passenger with high probability of survival:

In [ ]:
shap.plots.waterfall(shapley_values_test_extended[location_high_probability][:,1], 
                     max_display=15)

### [Scatter Plot](https://shap-lrjball.readthedocs.io/en/latest/example_notebooks/plots/scatter.html)

A scattter plot for one or more features shows the relationship between the feature value and the Shap value, along with a histogram of the frequency of the feature values.

In [ ]:
feat_to_show = shapley_top_10[0:4]

for feat in feat_to_show:
    shap.plots.scatter(shapley_values_test_extended[:, feat][:,1],
                       x_jitter=0)

Note: unlike a logistic regression model, Shap values are not linearly releated to feature values. This is because of the more flexible classification method in random forests.